In [2]:
import pandas as pd 
data = pd.read_csv("C:/Users/ntoure/Desktop/Python Projects/Transportation_Network.csv") 


In [3]:
data

,start,end,capacity,cost
0,gary,fra,900,39
1,gary,det,1200,14
2,gary,lan,600,11
3,gary,win,400,14
4,gary,stl,1700,16
5,gary,fre,1100,82
6,gary,laf,1000,8
7,clev,fra,900,27
8,clev,det,1200,9
9,clev,lan,600,12


In [30]:
list1 = data.start.unique();
list2 = data.end.unique();

L = list(set().union(list1, list2))
print(list1)
print(list2)
print(L)

NodeDict = { i : L[i] for i in range(0, len(L) ) }

['gary' 'clev' 'pitt']
['fra' 'det' 'lan' 'win' 'stl' 'fre' 'laf']
['stl', 'fre', 'fra', 'pitt', 'gary', 'laf', 'clev', 'lan', 'win', 'det']


In [49]:
start_nodes = []
end_nodes = []
capacities = []
costs = []

for index, row in data.iterrows():
    start_nodes.append(data['start'][index])
    end_nodes.append(data['end'][index])
    capacities.append(data['capacity'][index])
    costs.append(data['cost'][index])

In [60]:
for key, value in NodeDict.items():
    for i in range(len(start_nodes)):
        if start_nodes[i] == value:
            start_nodes[i] = key
    for i in range(len(end_nodes)):
        if end_nodes[i] == value:
            end_nodes[i] = key

In [61]:
print(start_nodes)
print(end_nodes)
print(capacities)
print(costs)

[4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3]
[2, 9, 7, 8, 0, 1, 5, 2, 9, 7, 8, 0, 1, 5, 2, 9, 7, 8, 0, 1, 5]
[900, 1200, 600, 400, 1700, 1100, 1000, 900, 1200, 600, 400, 1700, 1100, 1000, 900, 1200, 600, 400, 1700, 1100, 1000]
[39, 14, 11, 14, 16, 82, 8, 27, 9, 12, 9, 26, 95, 17, 24, 14, 17, 13, 28, 99, 20]


In [4]:
from __future__ import print_function
from ortools.graph import pywrapgraph
import time

def main():
  """Solving an Assignment Problem with MinCostFlow"""

  # Instantiate a SimpleMinCostFlow solver.
  min_cost_flow = pywrapgraph.SimpleMinCostFlow()
  # Define the directed graph for the flow.

  Node_Dict = {1 : "GARY ",
               2 : "CLEV ",
               3 : "PITT",
               4 : "FRA",
               5 : "DET",
               6 : "LAN",
               7 : "WIN",
               8 : "STL ",
               9 : "FRE",
               10 : "LAF"}
  
  start_nodes = [0, 0, 0] + [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,3, 3, 3, 3, 3, 3, 3] + [4, 5, 6, 7, 8, 9, 10]
  end_nodes =   [1, 2, 3,] + [4, 5, 6, 7, 8, 9, 10, 4, 5, 6, 7, 8, 9, 10, 4, 5, 6, 7, 8, 9, 10] + [11, 11, 11, 11, 11, 11, 11]
  capacities =  [1400, 2600, 2900] + [900, 1200, 600, 400, 1700, 1100, 1000,900, 1200, 600, 400, 1700, 1100, 1000, 900, 1200, 600, 400, 1700, 1100, 1000] + [900, 1200, 600, 400, 1700, 1100, 1000]
  costs  = ([0, 0, 0] + [39, 14, 11, 14, 16, 82, 8, 27, 9, 12, 9, 26, 95, 17, 24, 14, 17, 13, 28, 99, 20]
                + [0, 0, 0, 0, 0, 0, 0])
  # Define an array of supplies at each node.
  supplies = [6900, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -6900]
  source = 0 # Source number reference 
  sink = 11  # Sink number reference
  #tasks = 7   

  # Add each arc.
  for i in range(len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                                capacities[i], costs[i])

  # Add node supplies.

  for i in range(len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])
  # Find the minimum cost flow between node 0 and node 10.
  if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
    print('Total cost = ', min_cost_flow.OptimalCost())
    print()
    for arc in range(min_cost_flow.NumArcs()):

      # Can ignore arcs leading out of source or into sink.
      if min_cost_flow.Tail(arc)!=source and min_cost_flow.Head(arc)!=sink:

        # Arcs in the solution have a flow value of 1. Their start and end nodes
        # give an assignment of worker to task.

        if min_cost_flow.Flow(arc) > 0:
          print('Supply from %s assigned to Demand in %s :  Quantity = %d' % (
                Node_Dict[min_cost_flow.Tail(arc)],
                Node_Dict[min_cost_flow.Head(arc)],
                min_cost_flow.Flow(arc)))
  else:
    print('There was an issue with the min cost flow input.')
if __name__ == '__main__':
  start_time = time.clock()
  main()
  print()
  print("Time =", time.clock() - start_time, "seconds")

Total cost =  196200

Supply from GARY  assigned to Demand in FRE :  Quantity = 1100
Supply from GARY  assigned to Demand in LAF :  Quantity = 300
Supply from CLEV  assigned to Demand in DET :  Quantity = 1200
Supply from CLEV  assigned to Demand in LAN :  Quantity = 600
Supply from CLEV  assigned to Demand in WIN :  Quantity = 400
Supply from CLEV  assigned to Demand in LAF :  Quantity = 400
Supply from PITT assigned to Demand in FRA :  Quantity = 900
Supply from PITT assigned to Demand in STL  :  Quantity = 1700
Supply from PITT assigned to Demand in LAF :  Quantity = 300

Time = 0.0028976000003240188 seconds
